## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [1]:
import tensorflow as tf

In [0]:
import numpy as np

In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default
#tf.executing_eagerly()
tf.compat.v1.enable_eager_execution(
    config=None,
    device_policy=None,
    execution_mode=None
)

In [51]:
tf.__version__

'1.15.0'

### Collect Data

In [5]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('/gdrive/My Drive/Colab Notebooks/Residency6/prices.csv')

### Check all columns in the dataset

In [8]:
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


### Drop columns `date` and  `symbol`

In [0]:
data.drop(['date','symbol'],axis=1,inplace=True)

In [11]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [0]:
data['volume'] = data['volume']/1000000

In [16]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086


In [0]:
data = data[:1000]

In [32]:
data.shape

(1000, 5)

### Divide the data into train and test sets

In [0]:
X = data.drop("volume", axis=1)
y = data["volume"]

In [0]:
from sklearn.model_selection import train_test_split
test_size = 300 # taking 300 & 95 as training and test set sizes
seed = 7  # Random numbmer seeding for reapeatability of the code
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=test_size, random_state=seed)

#### Convert Training and Test Data to numpy float32 arrays


In [0]:
X_train = X_train.astype('float32')
y_train = y_train.astype('float32')
X_test = X_test.astype('float32')
y_test = y_test.astype('float32')

In [0]:
#as our tensorflow version is 1.15, we have to convert the y_train value to an array
y_train = np.array(y_train)

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer

In [0]:
transformer = Normalizer()
X_train = transformer.fit_transform(X_train)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
#We are initializing weights and Bias with Zero
w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [0]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y

3.Loss (Cost) Function [Mean square error]

In [0]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [63]:
#Train for 100 Steps
for i in range(100):
    
    w, b = train(X_train, y_train, w, b, learning_rate=0.01)
    print('Current Loss on iteration', i, 
          loss(y_train, prediction(X_train, w, b)).numpy())
    if i%5==0:
      print('Current Loss at consecutive 5th iteration', i, 
          loss(y_train, prediction(X_train, w, b)).numpy())

('Current Loss on iteration', 0, 273.9541)
('Current Loss at consecutive 5th iteration', 0, 273.9541)
('Current Loss on iteration', 1, 273.954)
('Current Loss on iteration', 2, 273.9541)
('Current Loss on iteration', 3, 273.95416)
('Current Loss on iteration', 4, 273.95416)
('Current Loss on iteration', 5, 273.95416)
('Current Loss at consecutive 5th iteration', 5, 273.95416)
('Current Loss on iteration', 6, 273.95416)
('Current Loss on iteration', 7, 273.95416)
('Current Loss on iteration', 8, 273.9541)
('Current Loss on iteration', 9, 273.95416)
('Current Loss on iteration', 10, 273.95416)
('Current Loss at consecutive 5th iteration', 10, 273.95416)
('Current Loss on iteration', 11, 273.95416)
('Current Loss on iteration', 12, 273.95416)
('Current Loss on iteration', 13, 273.95416)
('Current Loss on iteration', 14, 273.95416)
('Current Loss on iteration', 15, 273.9541)
('Current Loss at consecutive 5th iteration', 15, 273.9541)
('Current Loss on iteration', 16, 273.9541)
('Current Lo

### Get the shapes and values of W and b

In [61]:
print('Weights:\n', w.numpy())

('Weights:\n', array([[1.3771163],
       [1.3818748],
       [1.3629525],
       [1.3934754]], dtype=float32))


In [59]:
print('Bias:\n',b.numpy())

('Bias:\n', array([2.7578607], dtype=float32))


### Model Prediction on 1st Examples in Test Dataset

In [0]:
transformer = Normalizer()
X_test = transformer.fit_transform(X_test)

In [0]:
y = prediction(X_test[:1],w,b)

In [85]:
y.numpy()

array([[5.6101084]], dtype=float32)

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
iris = pd.read_csv('/gdrive/My Drive/Colab Notebooks/Residency6/Iris-2.csv')

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [236]:
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [0]:
iris_df = pd.get_dummies(iris,columns=['Species'])

In [238]:
iris_df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,1,5.1,3.5,1.4,0.2,1,0,0
1,2,4.9,3.0,1.4,0.2,1,0,0
2,3,4.7,3.2,1.3,0.2,1,0,0
3,4,4.6,3.1,1.5,0.2,1,0,0
4,5,5.0,3.6,1.4,0.2,1,0,0


### Splitting the data into feature set and target set

In [0]:
X = iris_df[['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']]
y = iris_df[['Species_Iris-setosa','Species_Iris-versicolor','Species_Iris-virginica']]

In [240]:
X.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [241]:
y.head()

,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


In [0]:
seed = 7  # Random numbmer seeding for reapeatability of the code
trainX, testX, trainy, testy = train_test_split(X,y,test_size=0.3, random_state=seed)

In [243]:
trainX.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
49,5.0,3.3,1.4,0.2
65,6.7,3.1,4.4,1.4
62,6.0,2.2,4.0,1.0
111,6.4,2.7,5.3,1.9
29,4.7,3.2,1.6,0.2


In [0]:
#trainy = tf.keras.utils.to_categorical(trainy, num_classes=3)
#testy = tf.keras.utils.to_categorical(testy, num_classes=3)

In [245]:
trainy.head()

,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
49,1,0,0
65,0,1,0
62,0,1,0
111,0,0,1
29,1,0,0


In [246]:
testX.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
149,5.9,3.0,5.1,1.8
84,5.4,3.0,4.5,1.5
40,5.0,3.5,1.3,0.3
66,5.6,3.0,4.5,1.5
106,4.9,2.5,4.5,1.7


###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
#Initialize Sequential model
model = tf.keras.models.Sequential()

#Normalize the data
#model.add(tf.keras.layers.BatchNormalization())

In [0]:
#Add INPUT layer
model.add(tf.keras.layers.Dense(4, input_shape = (4,), activation='sigmoid'))

In [0]:
#Add OUTPUT layer
#model.add(tf.keras.layers.Dense(3, input_shape = (4,), activation='softmax'))
model.add(tf.keras.layers.Dense(3, activation='softmax'))

In [0]:
#Compile the model
model.compile(optimizer='sgd', loss='categorical_crossentropy', 
              metrics=['accuracy'])

### Model Training 

In [251]:
model.fit(trainX,trainy,          
          validation_data=(testX,testy),
          epochs=300,
          batch_size=32) 

W0112 11:20:06.942454 140705068746624 training.py:503] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <type 'NoneType'>


Train on 105 samples, validate on 45 samples
Epoch 1/300
105/105 [==============================] - 0s 1ms/sample - loss: 1.1625 - acc: 0.3619 - val_loss: 1.2238 - val_acc: 0.2667
Epoch 2/300
105/105 [==============================] - 0s 99us/sample - loss: 1.1574 - acc: 0.3619 - val_loss: 1.2179 - val_acc: 0.2667
Epoch 3/300
105/105 [==============================] - 0s 94us/sample - loss: 1.1532 - acc: 0.3619 - val_loss: 1.2125 - val_acc: 0.2667
Epoch 4/300
105/105 [==============================] - 0s 95us/sample - loss: 1.1497 - acc: 0.3619 - val_loss: 1.2087 - val_acc: 0.2667
Epoch 5/300
105/105 [==============================] - 0s 94us/sample - loss: 1.1463 - acc: 0.3619 - val_loss: 1.2042 - val_acc: 0.2667
Epoch 6/300
105/105 [==============================] - 0s 99us/sample - loss: 1.1431 - acc: 0.3619 - val_loss: 1.2004 - val_acc: 0.2667
Epoch 7/300
105/105 [==============================] - 0s 87us/sample - loss: 1.1404 - acc: 0.3619 - val_loss: 1.1957 - val_acc: 0.2667
Epoc

### Model Prediction

In [0]:
input = np.array(testX[0:1])

In [259]:
pred = np.argmax(model.predict(input))
pred

2

### Save the Model

In [0]:
model.save('Iris_v1.h5')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?